In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [3]:
results = requests.get('https://whaletrips.org/map/js/getallcountries.php')

In [25]:
results.json()[0]

pd.DataFrame()

animals = [
    'Orca',
    'Glattwal',
    'Buckelwal',
    'Blauwal',
    'Pottwal',
    'Finnwal',
    'Beluga',
    'Narwal',
    'Zwergwal',
    'Grauwal'
]

season = [
    'frühjahr',
    'sommer',
    'herbst',
    'winter',
]

de2en = {
    'Orca': 'Orca', 
    'Glattwal': 'Right Whale', 
    'Buckelwal': 'Humpback Whale', 
    'Blauwal': 'Blue Whale', 
    'Pottwal': 'Sperm Whale', 
    'Finnwal': 'Fin Whale', 
    'Beluga': 'Beluga',
    'Narwal': 'Narwhal',
    'Zwergwal': 'Minke Whale',
    'Grauwal': 'Gray Whale',

    'frühjahr': 'spring',
    'sommer': 'summer',
    'herbst': 'autumn',
    'winter': 'winter'
}

columns = [
    de2en[x] + '_' + de2en[y] for x in animals for y in season
]

rows = [
    result['countryName'] for result in results.json()
]

animal2col = {
    x: i for i, x in enumerate(animals)
}

season2col = {
    x: i for i, x in enumerate(season)
}

values = np.zeros((len(rows), len(columns)))

for i, result in enumerate(results.json()):
    for x in result['filter']:
        name = x['name']
        col = 4*animal2col[name]
        seasons = x['properties']
        if type(seasons) == dict:
            seasons = x['properties'].values()
        for season in seasons:
            values[i][col+season2col[season['season']]] = 1

In [34]:
df = pd.DataFrame(
    index=rows,
    columns=columns,
)

df[columns] = values

# Fitness optimization

In [56]:
from scipy.optimize import minimize, LinearConstraint

In [53]:
def fitness(adj_array):
    adj_matrix = adj_array.reshape(len(df), len(df))
    external_sum = 0
    for x_i in values.T:
        internal_sum = 0
        for x_j in values.T:
            internal_sum += np.matmul(adj_matrix,x_j)
        external_sum += np.linalg.norm(x_i - internal_sum)
    return external_sum

In [61]:
constraints = []

# TODO: simplify to use (m, n) matrice instead

# Constraints on columns = 1
for i in range(len(df)):
    constraint_sums_one = np.zeros((len(df), len(df)))
    constraint_sums_one[:,i] = 1
    constraints.append(LinearConstraint(constraint_sums_one.reshape(1, -1), lb=1, ub=1))

# Constraints on a_i_j >= 0
for i in range(len(df)):
    for j in range(len(df)):
        constraint_postive = np.zeros((len(df), len(df)))
        constraint_postive[i,j] = 1
        constraints.append(LinearConstraint(constraint_postive.reshape(1, -1), lb=0))

In [66]:
minimize(fitness, np.identity(len(df)).reshape(-1), constraints=constraints)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 801.880601633848
       x: [ 4.613e-01  1.939e-02 ...  3.541e-02  5.195e-01]
     nit: 31
     jac: [ 5.225e+01  3.483e+01 ...  4.354e+01  1.742e+01]
    nfev: 13752
    njev: 31